# ANSWER KEY: Debug Drill 05 - Leaky Feature Engineering

**Bug:** Engineered features use future information:
- `days_to_cancel` - calculated from churn_date (future)
- `will_contact_support` - derived from cancel_reason (only known after cancellation)

**Key Lesson:** Feature engineering can introduce leakage. Audit every derived feature.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/shared/data/streamcart_customers.csv')

## The Bug (Colleague's Code)

In [ ]:
# ===== BUGGY CODE =====
df_eng = df.copy()

# "Helpful" feature engineering
df_eng['orders_per_month'] = df_eng['orders_last_30d'] / (df_eng['tenure_months'] + 1)  # OK
df_eng['login_trend'] = df_eng['logins_last_30d'] / (df_eng['logins_last_30d'].mean() + 1)  # OK

# LEAKY FEATURES:
df_eng['days_to_cancel'] = pd.to_datetime(df['churn_date']).sub(
    pd.to_datetime(df['snapshot_date'])
).dt.days.fillna(999)
# ^ Uses churn_date - we don't know this at prediction time!

df_eng['will_contact_support'] = (df['cancel_reason'].notna()).astype(int)
# ^ cancel_reason only exists AFTER they cancel!

features_buggy = [
    'tenure_months', 'orders_per_month', 'login_trend',
    'support_tickets_last_30d',
    'days_to_cancel',        # LEAKY
    'will_contact_support'   # LEAKY
]

X = df_eng[features_buggy].fillna(0)
y = df_eng['churn_30d']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model.fit(X_train, y_train)

auc_buggy = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print(f"Buggy model AUC: {auc_buggy:.3f}")
print("\nSuspiciously high...")

## Why This Is Wrong

| Feature | Problem |
|---------|--------|
| `days_to_cancel` | Uses `churn_date` which is FUTURE information |
| `will_contact_support` | Derived from `cancel_reason` which only exists AFTER cancellation |

**The leakage pattern:**
- `days_to_cancel` is small (or non-999) → customer will churn
- `will_contact_support = 1` → customer provided a cancel reason → they churned

These features ARE the answer, not predictors of it.

In [ ]:
# Investigate the leaky features
print("Leakage investigation:")
print("\ndays_to_cancel by churn status:")
print(df_eng.groupby('churn_30d')['days_to_cancel'].describe())

print("\nwill_contact_support by churn status:")
print(df_eng.groupby('churn_30d')['will_contact_support'].mean())
print("\n100% of churners have will_contact_support=1!")
print("This feature perfectly reveals the target.")

## The Fix

In [ ]:
# ===== FIXED CODE =====
# Only use features available at prediction time

df_fixed = df.copy()

# Safe engineered features (use only historical data)
df_fixed['orders_per_month'] = df_fixed['orders_last_30d'] / (df_fixed['tenure_months'] + 1)
df_fixed['login_trend'] = df_fixed['logins_last_30d'] / (df_fixed['logins_last_30d'].mean() + 1)
df_fixed['support_intensity'] = df_fixed['support_tickets_last_30d'] / (df_fixed['tenure_months'] + 1)

features_fixed = [
    'tenure_months',
    'orders_per_month',       # Safe - uses historical data
    'login_trend',            # Safe - uses historical data
    'support_tickets_last_30d',
    'support_intensity'       # Safe - uses historical data
]

X_fixed = df_fixed[features_fixed].fillna(0)
y_fixed = df_fixed['churn_30d']

X_train, X_test, y_train, y_test = train_test_split(X_fixed, y_fixed, test_size=0.2, random_state=42)
model_fixed = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model_fixed.fit(X_train, y_train)

auc_fixed = roc_auc_score(y_test, model_fixed.predict_proba(X_test)[:, 1])
print(f"Fixed model AUC: {auc_fixed:.3f}")
print(f"\nThis is realistic! The model uses only available information.")

In [ ]:
# Self-check
assert 0.55 < auc_fixed < 0.85, f"AUC {auc_fixed} seems off"
print("PASS: No leakage in engineered features!")

## Feature Engineering Audit Checklist

For every engineered feature, ask:

| Question | If No → |
|----------|--------|
| Would I have this at prediction time? | Remove it |
| Does it use any column with "date" in the name? | Audit carefully |
| Does it use target-related columns? | Almost certainly leaky |
| Does it use "reason" or "outcome" columns? | Likely post-event data |

## Completed Postmortem

### What happened:
- Colleague created features using future information (churn_date, cancel_reason)
- Model achieved unrealistic AUC by "peeking" at the answer

### Root cause:
- No temporal audit of feature engineering
- Easy to accidentally include post-event data when creating derived features

### How to prevent:
- For each feature, document: "Source column(s)" and "Available at prediction time?"
- Be especially careful with date-based and outcome-based columns
- If AUC jumps dramatically after feature engineering, audit new features